In [1]:
import cv2

import numpy as np

cap = cv2.VideoCapture("Hey Buddy, Can You Give Me a Hand.mp4")
 
width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

writer= cv2.VideoWriter('basicvideo.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 20, (width,height))

while True :
    ret,frame = cap.read() # frameler doğru okunduysa True (ret) okuyup frame değişkenine eşitliyodu
    
    frame_width = frame.shape[1] # en ve 
    frame_height = frame.shape[0]  # boylarını elde ediyorum . 
    frame_blob = cv2.dnn.blobFromImage(frame, 1/255, (416,416), swapRB=True, crop=False)
    labels = ["Spot | Boston Dynamics", "Mask"]
    # her bir label için farklı renkler vereceğim

    colors  = ["0, 255, 255", "0, 0, 255", "255, 0,0", "255, 255, 0 ", "0, 255, 0"]
    # bunlar str oldugu için bunları integera cevirmem gerekiyor,
    colors  = [np.array(color.split(",")).astype("int") for color in colors]
    # bunları tek bir array içine almak istiyorum 
    colors  = np.array(colors) 
    # colors matrisimdeki sayıları artıracağım ( parantezi içindeki sayı kadar sütun ve sayı ekler)
    colors =  np.tile(colors,(18,1))

    #--------------- artık algoritmamızı yazabiliriz

    model  = cv2.dnn.readNetFromDarknet("spot_yolov4.cfg","spot_yolov4_last.weights")

    # modelimin içindeki tüm layerlarımı çekmem gerekiyor detection için

    layers = model.getLayerNames()

    # benim işlem yapmak istediğim ise sadece detection yapılanlar
    # ancak benım bna ulaşmam gerekiyor 256 tane katman var ve hangisi output onu arıyorum 
    #  model.getUnconnectedOutLayers() bunu bulmama yarıyor ancak index 0'dan başladıgı için -1 yapmam gerekiyor

    output_layer = [layers[layer-1] for layer in model.getUnconnectedOutLayers()]

    """ output_layer =[]
    for i in model.getUnconnectedOutLayers() : 
        output_layer.append(int(i-1)) """

    # şimdi input olan blobumu verebilirim

    model.setInput(frame_blob) #  4 boyutlu format

    detection_layers = model.forward(output_layer)

        #############  NON-MAXIMUM SUPPRESSION - OPERATION 1 ###############
    ids_list = []
    boxes_list = []
    confidences_list = []
    ########################  END OF OPERATION 1 ########################
    for detection_layer in detection_layers: #3 arrayın içinde geziyorum
      for object_detection in detection_layer:

        scores =  object_detection[5:] # bu skorlardan en büyüğü benim nesnem olacak
        predicted_id = np.argmax(scores)
        confidence = scores[predicted_id] #GÜVEN SKORUMun tutulduğu yer

        if confidence > 0.35 : # güven skorum bu ddeğerden büyükse çizsin
          label = labels[predicted_id] # ancak bu hangi label onu bulmam gerekiyor
          bounding_box = object_detection[0:4] * np.array([frame_width, frame_height, frame_width, frame_height]) #en ve boy ile genişletiyorum 
            # ilk 4 değerim benım için yeterli değil bu yüzden w,h değerleri ile çarpmıp büyütmem gerekiyor
          (box_center_x, box_center_y, box_width, box_height) = bounding_box.astype("int")
            #  dikdörtgen çizebilmem için buna ihtiyacım var 

          start_x = int(box_center_x - (box_width/2)) #baslangıc ve
          start_y = int(box_center_y - (box_height/2)) 

          #############      NON-MAXIMUM SUPPRESSION - OPERATION 2 ###############   
          # listelerin içini dolduracağım
          ids_list.append(predicted_id)
          confidences_list.append(float(confidence))  # float alıyorum değerlerimin büyük çıkmasını istemiyorum
          boxes_list.append([start_x,start_y, int(box_width),int(box_height)])

          #############--------------- END OF OPERATION 2 ------- ###############    
            
            #############  NON-MAXIMUM SUPPRESSION - OPERATION - 3 ###############          
    # en yüksek güvenirliliğe sahip dikdörtgenlerin id lerini döndürür

    max_ids = cv2.dnn.NMSBoxes(boxes_list, confidences_list, 0.5, 0.4) # 0,5 güven skoru, 0.4 threshold değeri
    for maxId in max_ids:
        box = boxes_list[maxId]
        start_x = box[0]
        start_y = box[1]
        box_width = box[2]
        box_height = box[3]

         # listemin içinden tahminlerimi alıyorum
        predicted_id = ids_list[maxId]
        label        = labels[predicted_id]
        confidence   = confidences_list[maxId]

    #############------------- END OF OPERATION 3  ---------###############        



        end_x = start_x + box_width                    # bitiş noktlarını girmem gerekiyor


        end_y = start_y + box_height

        box_color = colors[predicted_id]    # herbir nesne için ayrı kutular çizmiş olacağım

        box_color = [int(each) for each in box_color] #  

        label = "{}: {:.2f}%".format(label, confidence * 100)
        print("predicted object {}".format(label))

        cv2.rectangle(frame,(start_x,start_y), (end_x,end_y),box_color,2)
        # başlangıc ve bitiş noktalarımı girdim en son kalınlıgını 
        cv2.putText(frame,label,(start_x,start_y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,box_color,2)
            # neresinde konumlansın bu , onu belirliyorum 
    writer.write(frame)
    cv2.imshow("Detection Window", frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

        
cap.release()

writer.release()
cv2.destroyAllWindows()




predicted object Spot | Boston Dynamics: 67.71%
predicted object Spot | Boston Dynamics: 71.93%
predicted object Spot | Boston Dynamics: 75.44%
predicted object Spot | Boston Dynamics: 75.63%
predicted object Spot | Boston Dynamics: 71.62%
predicted object Spot | Boston Dynamics: 70.29%
predicted object Spot | Boston Dynamics: 69.83%
predicted object Spot | Boston Dynamics: 63.58%
predicted object Spot | Boston Dynamics: 65.37%
predicted object Spot | Boston Dynamics: 70.23%
predicted object Spot | Boston Dynamics: 56.55%
predicted object Spot | Boston Dynamics: 55.57%
predicted object Spot | Boston Dynamics: 57.65%
predicted object Spot | Boston Dynamics: 79.87%
predicted object Spot | Boston Dynamics: 79.41%
predicted object Spot | Boston Dynamics: 87.22%
predicted object Spot | Boston Dynamics: 87.26%
predicted object Spot | Boston Dynamics: 87.15%
predicted object Spot | Boston Dynamics: 85.28%
predicted object Spot | Boston Dynamics: 85.08%
predicted object Spot | Boston Dynamics: